<a href="https://colab.research.google.com/github/eunzzae/dacon_Ecommerce/blob/main/Dacon_%EC%9D%B4%EC%BB%A4%EB%A8%B8%EC%8A%A4_%EA%B3%A0%EA%B0%9D%EB%B6%84%EB%A5%98_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [12]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [13]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import rc

plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] =False

# 데이터 불러오기

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# 각자 작업 환경에 맞는 경로를 지정해주세요. Google Colab과 Jupyter환경에서 경로가 다를 수 있습니다.
base_path = '/content/drive/MyDrive/Colab Notebooks/data/dacon_ecomerce/'
df_customer = pd.read_csv(base_path+'Customer_info.csv')
df_discount = pd.read_csv(base_path+'Discount_info.csv')
df_marketing = pd.read_csv(base_path+'Marketing_info.csv')
df_onlinesales = pd.read_csv(base_path+'Onlinesales_info.csv')
df_tax = pd.read_csv(base_path+'Tax_info.csv')
df_merge = pd.read_csv(base_path+'Merge.csv')

In [16]:
df_merge.head()

,고객ID,거래ID,거래날짜,제품ID,제품카테고리,수량,평균금액,배송료,쿠폰상태,성별,고객지역,가입기간,월,쿠폰코드,할인율,GST
0,USER_1358,Transaction_0000,2019-01-01,Product_0981,Nest-USA,1,153.71,6.5,Used,남,Chicago,12,1,ELEC10,10.0,0.10
1,USER_1358,Transaction_0001,2019-01-01,Product_0981,Nest-USA,1,153.71,6.5,Used,남,Chicago,12,1,ELEC10,10.0,0.10
2,USER_1358,Transaction_0002,2019-01-01,Product_0904,Office,1,2.05,6.5,Used,남,Chicago,12,1,OFF10,10.0,0.10
3,USER_1358,Transaction_0003,2019-01-01,Product_0203,Apparel,5,17.53,6.5,Not Used,남,Chicago,12,1,SALE10,10.0,0.18
4,USER_1358,Transaction_0003,2019-01-01,Product_0848,Bags,1,16.50,6.5,Used,남,Chicago,12,1,AIO10,10.0,0.18


# RFM 분석
* Recency(최근 구매) : 고객이 마지막으로 구매한 시점
* Frequency(구매 빈도) : 특정 기간 동안 얼마나 자주 구매했는지
* Monetary(구매 금액) : 고객이 구매에 지출하는 총 금액

In [17]:
# 고객ID이 가장 최근에 구매한 날짜
monthly_purchase_frequency = df_merge.groupby('고객ID')['거래날짜'].max().reset_index()
monthly_purchase_frequency.columns = ['고객ID', '최근구매일']
monthly_purchase_frequency.head()

,고객ID,최근구매일
0,USER_0000,2019-09-15
1,USER_0001,2019-11-02
2,USER_0002,2019-10-19
3,USER_0003,2019-12-14
4,USER_0004,2019-09-15


In [20]:
# 고객ID당 총 구매횟수
quantiry_sum_customer = df_merge.groupby('고객ID')['수량'].sum().reset_index()
quantiry_sum_customer.columns = ['고객ID', '수량합']
quantiry_sum_customer.head()

,고객ID,구매수량
0,USER_0000,3
1,USER_0001,342
2,USER_0002,209
3,USER_0003,21
4,USER_0004,56


In [24]:
# 고객ID당 월별 구매횟수

# 거래날짜를 datetime 타입으로 변환
df_merge['거래날짜'] = pd.to_datetime(df_merge['거래날짜'])

# 고객별, 월별 구매 빈도 계싼
monthly_purchase_frequency = df_merge.groupby(['고객ID','월']).size().reset_index(name='구매횟수')
monthly_purchase_frequency.head()

,고객ID,월,구매횟수
0,USER_0000,9,2
1,USER_0001,3,31
2,USER_0001,11,29
3,USER_0002,6,17
4,USER_0002,10,6


In [30]:
# 고객ID당 총 지출한 금액
df_merge['총 금액'] = df_merge['수량']* df_merge['평균금액']
total_spent_customer = df_merge.groupby('고객ID')['총 금액'].sum().reset_index()
total_spent_customer.columns = ['고객ID', '총 지출금액']
total_spent_customer.head()

,고객ID,총 지출금액
0,USER_0000,30.99
1,USER_0001,13834.90
2,USER_0002,1442.12
3,USER_0003,1360.07
4,USER_0004,1442.47


In [32]:
# 고객ID당 월별 총 지출한 금액
monthly_total_spent_customer = df_merge.groupby(['고객ID', '월'])['총 금액'].sum().reset_index()
monthly_total_spent_customer.columns = ['고객ID', '월', '총 지출금액']
monthly_total_spent_customer.head()

,고객ID,월,총 지출금액
0,USER_0000,9,30.99
1,USER_0001,3,11216.05
2,USER_0001,11,2618.85
3,USER_0002,6,777.03
4,USER_0002,10,665.09


# 코호트 분석
* 시간 기반
: 특정 시간(월, 분기) 가입하거나 첫 구매한 고객 기준  
=> 특정 시간대 유입된 고객의 행동 패턴, 생애 가치 평가

* 행동 기반
: 특정 행동(첫 구매 제품유형, 이용한 프로모션 종류 등) 기준  
=> 특정 행동을 한 고객의 장기적 가치, 이탈률 파악